# Title: IP Address Explorer
LogAnalytics

## Description:
Series of modules designed to help get a better understanding of the contents of a process-based alert.


<a id='toc'></a>
## Table of Contents
- [Setup and Authenticate](#setup)

- [Get Alerts List](#getalertslist)
- [Choose an Alert to investigate](#enteralertid)
  - [Extract Properties and entities from alert](#extractalertproperties)
  - [Entity Graph](#entitygraph)
- [Related Alerts](#related_alerts)
- [Session Process Tree](#processtree)
  - [Process Timeline](#processtimeline)
- [Other Process on Host](#process_clustering)
- [Check for IOCs in Commandline](#cmdlineiocs)
  - [VirusTotal lookup](#virustotallookup)
- [Alert command line - Occurrence on other hosts in subscription](#cmdlineonotherhosts)
- [Host Logons](#host_logons)
  - [Alert Account](#logonaccount)
  - [Failed Logons](#failed_logons)
- [Appendices](#appendices)
  - [Saving data to Excel](#appendices)
  
- Linux machine SSH Brute force alert
- Network logs - traced to Windows machine
- Windows machine
  - View logons
  - See RDP logon
  - Get process session for suspicious logon
  - Process tree shows FTP downloading tools
  - Lookup domain in TI to confirm

### Import Python Packages

In [ ]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

import numpy as np
from IPython import get_ipython
from IPython.display import display, HTML
import ipywidgets as widgets

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import msticpy.sectools as sectools
import msticpy.asitools as asi
import msticpy.asitools.kql as qry
import msticpy.asitools.nbdisplay as asidisp

WIDGET_DEFAULTS = {'layout': widgets.Layout(width='95%'),
                   'style': {'description_width': 'initial'}}
HTML('''
    <script type="text/javascript">
        IPython.notebook.kernel.execute("nb_query_string='".concat(window.location.search).concat("'"));
    </script>
    ''');

### Enter or confirm WorkspaceId
To find your Workspace Id go to [Log Analytics](#https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.

In [ ]:
available_workspaces = {'Contoso77':'802d39e1-9d70-404d-832c-2de5e2478eda', 
                        'MSTICLinux':'06dc719f-5dad-47e9-b5af-07d84a0bda4e',
                        'ASIHuntOMSWorkspaceV4': '52b1ab41-869e-4138-9e40-2a4457f09bf0',
                        'ASIHuntOMSWorkspaceV5': '4ca7b24a-6e8f-4540-a8ce-1a80c2948c37',
                        'Rome ILDC - Detection E2E Tests Stage': '3eb61071-5dcd-4db3-94fa-0091a69b7359'}
select_ws = asi.SelectString(description='Select workspace :',
                             item_dict=available_workspaces)

select_ws.display()

In [ ]:
import os
from msticpy.asitools.asiconfig import WorkspaceConfig
ws_config_file = 'config.json'
try:
    ws_config = WorkspaceConfig(ws_config_file)
    print('Found config file')
    for cf_item in ['tenant_id', 'subscription_id', 'resource_group', 'workspace_id', 'workspace_name']:
        print(cf_item, ws_config[cf_item])
except:
    ws_config = None

ws_id = asi.GetEnvironmentKey(env_var='WORKSPACE_ID',
                              prompt='Log Analytics Workspace Id:')
if ws_config:
    ws_id.value = ws_config['workspace_id']
ws_id.display()

### Authenticate to Log Analytics
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. 
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. 
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)
```

In [ ]:
try:
    WORKSPACE_ID = select_ws.value
except NameError:
    try:
        WORKSPACE_ID = ws_id.value
    except NameError:
        WORKSPACE_ID = None
    
if not WORKSPACE_ID:
    raise ValueError('No workspace selected.')

asi.kql.load_kql_magic()

%kql loganalytics://code().workspace(WORKSPACE_ID)


<a id='get_ip'></a>[Contents](#toc)
# Enter the IP Address or paste an IP entity.
Set the query time window

In [ ]:
ip_text = widgets.Text(description='Enter the IP Address to search for:', **WIDGET_DEFAULTS)
display(ip_text)

# Internal Address Flow

## Try to find a host with this address

In [ ]:
aznet_query = '''
AzureNetworkAnalytics_CL 
| where PrivateIPAddresses_s has \'{address}\' 
or PublicIPAddresses_s has \'{address}\'
| where ResourceType == 'NetworkInterface'
| top 1 by TimeGenerated desc
| extend Computer = split(VirtualMachine_s, '/')[-1]
| project Computer, PrivateIPAddresses = PrivateIPAddresses_s, 
    PublicIPAddresses = PublicIPAddresses_s
'''.format(address=ip_text.value.strip())
%kql -query aznet_query
az_net_df = _kql_raw_result_.to_dataframe()


oms_heartbeat_query = '''
Heartbeat
| where ComputerIP == \'{address}\'
| top 1 by TimeGenerated desc nulls last
| project Computer, ComputerIP, OSType, OSMajorVersion, OSMinorVersion, ResourceId, RemoteIPCountry, 
RemoteIPLatitude, RemoteIPLongitude, SourceComputerId
'''.format(address=ip_text.value.strip())
%kql -query oms_heartbeat_query
oms_heartbeat_df = _kql_raw_result_.to_dataframe()

if len(oms_heartbeat_df == 1):
    host_name = oms_heartbeat_df['Computer'].loc[0]
elif len(az_net_df == 1):
    host_name = az_net_df['Computer'].loc[0]
else:
    host_name = 'unknown'
    

In [ ]:
# Get the host entity and add this IP and system info to the entities
from msticpy.sectools.geoip import GeoLiteLookup
iplocation = GeoLiteLookup()

try:
    if not inv_host_entity:
        inv_host_entity = asi.Host()
        
except NameError:
    inv_host_entity = asi.Host()
    inv_host_entity.HostName = host_name
    
src_ip_entity = asi.IpAddress(Address=ip_text.value.strip())
iplocation.lookup_ip(ip_entity=src_ip_entity)

def convert_to_ip_entities(ip_str):
    ip_entities = []
    if ip_str:
        if ',' in ip_str:
            addrs = ip_str.split(',')
        elif ' ' in ip_str:
            addrs = ip_str.split(' ')
        else:
            addrs = [ip_str]
        for addr in addrs:
            ip_entity = asi.IpAddress()
            ip_entity.Address = addr.strip()
            iplocation.lookup_ip(ip_entity=ip_entity)
            ip_entities.append(ip_entity)
    return ip_entities

# Add this information to our inv_host_entity
if len(az_net_df) == 1:
    priv_addr_str = az_net_df['PrivateIPAddresses'].loc[0]
    inv_host_entity.properties['private_ips'] = convert_to_ip_entities(priv_addr_str)

    pub_addr_str = az_net_df['PublicIPAddresses'].loc[0]
    inv_host_entity.properties['public_ips'] = convert_to_ip_entities(pub_addr_str)

retrieved_address = [ip.Address for ip in inv_host_entity.properties['public_ips']]
if len(oms_heartbeat_df) == 1:
    if oms_heartbeat_df['ComputerIP'].loc[0]:
        oms_address = oms_heartbeat_df['ComputerIP'].loc[0]
        if oms_address not in retrieved_address:
            ip_entity = asi.IpAddress()
            ip_entity.Address = oms_address
            iplocation.lookup_ip(ip_entity=ip_entity)
            inv_host_entity.properties['public_ips'].append(ip_entity)
        
    inv_host_entity.OSFamily = oms_heartbeat_df['OSType'].loc[0]
    inv_host_entity.AdditionalData['OSMajorVersion'] = oms_heartbeat_df['OSMajorVersion'].loc[0]
    inv_host_entity.AdditionalData['OSMinorVersion'] = oms_heartbeat_df['OSMinorVersion'].loc[0]
    inv_host_entity.AdditionalData['SourceComputerId'] = oms_heartbeat_df['SourceComputerId'].loc[0]

print('Updated Host Entity\n')
print(inv_host_entity)
print(src_ip_entity)

### Geo-mapping function definition

In [ ]:
import folium
from folium.plugins import MarkerCluster
from numbers import Number
import warnings

def create_ip_map():
    folium_map = folium.Map(zoom_start=7, tiles=None, width='100%', height='100%')
    folium_map.add_tile_layer(name='IPEvents')
    return folium_map

def add_ip_cluster(folium_map, ip_entities, alert=None, **icon_props):
    if not folium_map:
        folium_map = create_ip_map()
    marker_cluster = MarkerCluster()

    for ip_entity in ip_entities:
        if not (isinstance(ip_entity.Location.Latitude, Number) and
                isinstance(ip_entity.Location.Longitude, Number)):
            warnings.warn("Invalid location information for IP: " + ip_entity.Address,
                          RuntimeWarning)
            continue
        loc_props = ', '.join([f'{key}={val}' for key, val in 
                               ip_entity.Location.properties.items() if val])
        popup_text = "{loc_props}<br>{IP}".format(IP=ip_entity.Address,
                                                  loc_props=loc_props)
        tooltip_text = '{City}, {CountryName}'.format(**ip_entity.Location.properties)
        if alert:
            popup_text = f'{popup_text}<br>{alert.AlertName}'
        if ip_entity.AdditionalData:
            addl_props = ', '.join([f'{key}={val}' for key, val in 
                                    ip_entity.AdditionalData.items() if val])
            popup_text = f'{popup_text}<br>{addl_props}'
            tooltip_text = f'{tooltip_text}, {addl_props}'
        marker = folium.Marker(
            location = [ip_entity.Location.Latitude, ip_entity.Location.Longitude],
            popup=popup_text,
            tooltip=tooltip_text,
            icon=folium.Icon(**icon_props)
        )
        marker_cluster.add_child(marker)

    folium_map.add_child(marker_cluster)
    return folium_map

ip_loc_map = create_ip_map()
icon_props = {'color': 'blue', 'icon': 'crosshairs', 'prefix': 'fa'}
ip_loc_map = add_ip_cluster(folium_map=ip_loc_map,
                            ip_entities=[src_ip_entity],
                            **icon_props)
display(HTML('<h3>Location of IP Address in alert</h3>'))
display(ip_loc_map)

### Reverse Lookup and WhoIs

In [ ]:
# reverse DNS lookup
from dns import reversename, resolver
from ipwhois import IPWhois
for src_ip_entity in alert_ip_entities:
    print('IP:', src_ip_entity.Address)
    print('-'*50)
    
    print('Reverse Name Lookup.')
    rev_name = reversename.from_address(src_ip_entity.Address)
    
    print(rev_name)
    try:
        rev_dns = str(resolver.query(rev_name, 'PTR'))
        display(rev_dns)
    except:
        print('No reverse addr result')
        pass

    print('\nWhoIs Lookup.')
    whois = IPWhois(src_ip_entity.Address)
    whois_result = whois.lookup_whois()
    if whois_result:
        display(whois_result)
    else:
        print('No whois result')

<a id='check_alert_cd'></a>[Contents](#toc)
## Check the IP Address for known C2 addresses

In [ ]:
# Lookup in BYOTI or VT (or IPRep)

<a id='comms_to_other_hosts'></a>[Contents](#toc)
## Check Communications with Other Hosts

In [ ]:
# Azure Network Analytics Base Query
az_net_analytics_query =r'''
AzureNetworkAnalytics_CL 
| where SubType_s == 'FlowLog'
| where FlowStartTime_t >= datetime({start})
| where FlowEndTime_t <= datetime({end})
| project TenantId, TimeGenerated, 
    FlowStartTime = FlowStartTime_t, 
    FlowEndTime = FlowEndTime_t, 
    FlowIntervalEndTime = FlowIntervalEndTime_t, 
    FlowType = FlowType_s,
    ResourceGroup = split(VM_s, '/')[0],
    VMName = split(VM_s, '/')[1],
    VMIPAddress = VMIP_s, 
    PublicIPs = extractall(@"([\d\.]+)[|\d]+", dynamic([1]), PublicIPs_s),
    L4Protocol = L4Protocol_s, 
    L7Protocol = L7Protocol_s, 
    DestPort = DestPort_d, 
    FlowDirection = FlowDirection_s,
    AllowedOutFlows = AllowedOutFlows_d, 
    AllowedInFlows = AllowedInFlows_d,
    DeniedInFlows = DeniedInFlows_d, 
    DeniedOutFlows = DeniedOutFlows_d,
    RemoteRegion = AzureRegion_s,
    VMRegion = Region_s
| where {where_clause}
'''

ip_q_times = asi.QueryTime(label='Set time bounds for network queries',
                           units='hour', max_before=48, before=5, after=1, 
                           max_after=24)
ip_q_times.display()

### Query Flows by Host IP Addresses

In [ ]:
all_inv_host_ips = inv_host_entity.private_ips + inv_host_entity.public_ips
host_ips = {'\'{}\''.format(i.Address) for i in all_inv_host_ips}
inv_host_ip_list = ','.join(host_ips)

az_ip_where = f'''
VMIPAddress in ({inv_host_ip_list}) and 
(AllowedOutFlows > 0 or AllowedInFlows > 0)'''

az_net_query_byip = az_net_analytics_query.format(where_clause=az_ip_where,
                                                  start = ip_q_times.start,
                                                  end = ip_q_times.end)
%kql -query az_net_query_byip
az_net_comms_df = _kql_raw_result_.to_dataframe()
az_net_comms_df

### Query Flows by HostName (preferred)

In [ ]:
# restrict to host and only include allowed flows
az_host_where = f'''
VMName == tolower(\'{inv_host_entity.HostName}\') and 
(AllowedOutFlows > 0 or AllowedInFlows > 0)
'''
az_net_query_byhost = az_net_analytics_query.format(where_clause=az_host_where,
                                                    start = ip_q_times.start,
                                                    end = ip_q_times.end)
%kql -query az_net_query_byhost
az_net_comms_df = _kql_raw_result_.to_dataframe()
az_net_comms_df

### Time and Protocol Distribution

In [ ]:
az_net_comms_df['TotalAllowedFlows'] = az_net_comms_df['AllowedOutFlows'] + az_net_comms_df['AllowedInFlows']
sns.catplot(x="L7Protocol", y="TotalAllowedFlows", col="FlowDirection", data=az_net_comms_df)
sns.relplot(x="FlowStartTime", y="TotalAllowedFlows", col="FlowDirection", kind="line", hue="L7Protocol", data=az_net_comms_df)

### Interactive Flow Timeline

In [ ]:
az_net_comms_df['ExtIPs'] = az_net_comms_df.apply(lambda x: ', '.join(list(x.PublicIPs)) if
                                                  x.PublicIPs else None, axis=1)
asidisp.display_timeline(data=az_net_comms_df.query('AllowedOutFlows > 0 or AllowedInFlows > 0'),
                         title='Network Flows',
                         time_column='FlowStartTime',
                         source_columns=['FlowType', 'ExtIPs', 'L7Protocol', 'FlowDirection'],
                         height=300)

## GeoLocation Mapping

In [ ]:
ip_locs_in = set()
ip_locs_out = set()
for _, row in az_net_comms_df.iterrows():
    for ip in row.PublicIPs:
        if ip in ip_locs_in or ip in ip_locs_out:
            continue
        ip_entity = asi.IpAddress(Address=ip)
        iplocation.lookup_ip(ip_entity=ip_entity)
        if not ip_entity.Location:
            continue
        ip_entity.AdditionalData['protocol'] = row.L7Protocol
        if row.FlowDirection == 'I':
            ip_locs_in.add(ip_entity)
        else:
            ip_locs_out.add(ip_entity)
            
flow_map = create_ip_map()
display(HTML('<h3>External IP Addresses communicating with host</h3>'))
display(HTML('Numbered circles indicate multiple items - click to expand'))
display(HTML('Location markers: Blue = outbound, Purple = inbound'))


icon_props = {'color': 'purple'}
flow_map = add_ip_cluster(folium_map=flow_map,
                            ip_entities=ip_locs_in,
                            **icon_props)
icon_props = {'color': 'blue'}
flow_map = add_ip_cluster(folium_map=flow_map,
                            ip_entities=ip_locs_out,
                            **icon_props)
display(flow_map)


### Include Denied Flows - who's trying to get in from where?

In [ ]:
az_host_where = f'''
VMName == tolower(\'{inv_host_entity.HostName}\')
'''
az_net_query_byhost = az_net_analytics_query.format(where_clause=az_host_where,
                                                    start = ip_q_times.start,
                                                    end = ip_q_times.end)

%kql -query az_net_query_byhost
az_net_comms_block_df = _kql_raw_result_.to_dataframe()
az_net_comms_block_df

In [ ]:
ip_all = set()
ip_locs_in_allow = set()
ip_locs_out_allow = set()
ip_locs_in_deny = set()
ip_locs_out_deny = set()
for _, row in az_net_comms_block_df.iterrows():
    if not row.PublicIPs:
        continue
    for ip in row.PublicIPs:
        if ip in ip_all:
            continue
        ip_all.add(ip)
        ip_entity = asi.IpAddress(Address=ip)
        iplocation.lookup_ip(ip_entity=ip_entity)
        if not ip_entity.Location:
            warnings.warn("No location information for IP: " + ip,
                          RuntimeWarning)
            continue
        ip_entity.AdditionalData['protocol'] = row.L7Protocol
        if row.FlowDirection == 'I':
            if row.AllowedInFlows > 0:
                ip_locs_in_allow.add(ip_entity)
            elif row.DeniedInFlows > 0:
                ip_locs_in_deny.add(ip_entity)
        else:
            if row.AllowedOutFlows > 0:
                ip_locs_out_allow.add(ip_entity)
            elif row.DeniedOutFlows > 0:
                ip_locs_out_deny.add(ip_entity)
            
flow_map = create_ip_map()
display(HTML('<h3>External IP Addresses Blocked and Allowed communicating with host</h3>'))
display(HTML('Numbered circles indicate multiple items - click to expand.'))
display(HTML('Location markers: Blue = outbound, Purple = inbound, Red = in denied, Cyan = out denied.'))


icon_props = {'color': 'purple'}
flow_map = add_ip_cluster(folium_map=flow_map,
                            ip_entities=ip_locs_in_allow,
                            **icon_props)
icon_props = {'color': 'blue'}
flow_map = add_ip_cluster(folium_map=flow_map,
                            ip_entities=ip_locs_out_allow,
                            **icon_props)
icon_props = {'color': 'red'}
flow_map = add_ip_cluster(folium_map=flow_map,
                            ip_entities=ip_locs_in_deny,
                            **icon_props)
icon_props = {'color': 'cyan'}
flow_map = add_ip_cluster(folium_map=flow_map,
                            ip_entities=ip_locs_out_deny,
                            **icon_props)
display(flow_map)


## Flows to/from External IPs that are blocked in other cases

In [ ]:
asidemo_query = '''
let denied_flows = materialize(AzureNetworkAnalytics_CL 
| where SubType_s == 'FlowLog'
| where TimeGenerated >= ago(1d)
| project TenantId, TimeGenerated, 
    FlowStartTime = FlowStartTime_t, 
    FlowEndTime = FlowEndTime_t, 
    FlowIntervalEndTime = FlowIntervalEndTime_t, 
    FlowType = FlowType_s,
    ResourceGroup = split(VM_s, '/')[0],
    VMName = tostring(split(VM_s, '/')[1]),
    VMIPAddress = VMIP_s,
    PublicIPs = extractall(@"([\d\.]+)[|\d]+", dynamic([1]), PublicIPs_s),
    L4Protocol = L4Protocol_s, 
    L7Protocol = L7Protocol_s, 
    DestPort = DestPort_d, 
    FlowDirection = FlowDirection_s,
    AllowedOutFlows = AllowedOutFlows_d, 
    AllowedInFlows = AllowedInFlows_d,
    DeniedInFlows = DeniedInFlows_d, 
    DeniedOutFlows = DeniedOutFlows_d,
    RemoteRegion = AzureRegion_s,
    VMRegion = Region_s 
| where DeniedInFlows > 0 or DeniedOutFlows > 0
| mvexpand PublicIPs
| extend PublicIP = tostring(PublicIPs)
| project-away PublicIPs
| summarize denied_in = sum(DeniedInFlows), denied_out = sum(DeniedOutFlows) by FlowDirection, PublicIP, L7Protocol, VMName
| project denied_in, denied_out, d_FlowDirection=FlowDirection, PublicIP, d_L7Protocol=L7Protocol, d_VMName=VMName
);
denied_flows
| join (
    AzureNetworkAnalytics_CL
    | where TimeGenerated >= ago(1d)
    | project 
        ResourceGroup = split(VM_s, '/')[0],
        VMName = tostring(split(VM_s, '/')[1]),
        VMIPAddress = VMIP_s,
        PublicIPs = extractall(@"([\d\.]+)[|\d]+", dynamic([1]), PublicIPs_s),
        L7Protocol = L7Protocol_s, 
        DestPort = DestPort_d, 
        FlowDirection = FlowDirection_s,
        AllowedOutFlows = AllowedOutFlows_d, 
        AllowedInFlows = AllowedInFlows_d
    | where AllowedInFlows > 0 or AllowedOutFlows > 0
    | mvexpand PublicIPs
    | extend PublicIP = tostring(PublicIPs) 
    | project-away PublicIPs
    | project a_FlowDirection=FlowDirection, PublicIP, a_L7Protocol=L7Protocol, a_VMName=VMName, AllowedInFlows, AllowedOutFlows
) on PublicIP 
| summarize allowed_in = sum(AllowedInFlows), allowed_out = sum(AllowedOutFlows) by PublicIP, d_FlowDirection, d_L7Protocol, d_VMName, denied_in, denied_out,a_FlowDirection, a_L7Protocol, a_VMName
| summarize a_L7Protocols = count(a_L7Protocol), allowed_in_t = sum(allowed_in), allowed_out_t = sum(allowed_out) by PublicIP, a_VMName
'''

%kql -query asidemo_query
asidemo_df = _kql_raw_result_.to_dataframe()


ip_all = set()
ip_locs_in_allow = set()
ip_locs_out_allow = set()

for _, row in asidemo_df.iterrows():
    if not row.PublicIP:
        continue
    ip = row.PublicIP
    if ip in ip_all:
        continue
    ip_all.add(ip)
    ip_entity = asi.IpAddress(Address=ip)
    iplocation.lookup_ip(ip_entity=ip_entity)
    if not ip_entity.Location:
        warnings.warn("No location information for IP: " + ip,
                      RuntimeWarning)
        continue
    ip_entity.AdditionalData['protocols'] = row.a_L7Protocols
    if row.allowed_in_t > 0:
        ip_locs_in_allow.add(ip_entity)
    elif row.allowed_out_t > 0:
        ip_locs_out_allow.add(ip_entity)
            
flow_map_asi = create_ip_map()
display(HTML('<h3>External IP Addresses Allowed communicating with host</h3>'))
display(HTML('Numbered circles indicate multiple items - click to expand.'))
display(HTML('Location markers: Red = in, Purple = out.'))


icon_props = {'color': 'red'}
flow_map_asi = add_ip_cluster(folium_map=flow_map_asi,
                            ip_entities=ip_locs_in_allow,
                            **icon_props)
icon_props = {'color': 'purple'}
flow_map_asi = add_ip_cluster(folium_map=flow_map_asi,
                            ip_entities=ip_locs_out_allow,
                            **icon_props)

display(flow_map_asi)


In [ ]:
dns_query =r'''
DnsEvents
| where ClientIP in ({ip_list})
'''.format(ip_list=inv_host_ip_list)

%kql -query dns_query
dns_df = _kql_raw_result_.to_dataframe()
dns_df

In [ ]:
dns_query =r'''
DnsEvents
| where IPAddresses has ('{src_ip}')
'''.format(src_ip=ip_text.value.strip())

%kql -query dns_query
dns_df = _kql_raw_result_.to_dataframe()
dns_df

In [ ]:
TBD

### Have any other hosts been communicating with this address?

In [ ]:
ip_q_times = asi.QueryTime(units='day', max_before=10, before=3, after=1, max_after=10)
ip_q_times.display()

In [ ]:
target_ip = ip_text.value.strip()
az_ip_where = f'PublicIPs has \'{target_ip}\''

az_net_query_by_pub_ip = az_net_analytics_query.format(where_clause=az_ip_where,
                                                       start = ip_q_times.start,
                                                       end = ip_q_times.end)

%kql -query az_net_query_by_pub_ip
az_net_ext_comms_df = _kql_raw_result_.to_dataframe()
az_net_ext_comms_df.groupby(['VMName', 'L7Protocol'])['AllowedOutFlows','AllowedInFlows','DeniedInFlows','DeniedOutFlows'].sum()

In [ ]:
sns.set()
from matplotlib import MatplotlibDeprecationWarning
warnings.simplefilter("ignore", category=MatplotlibDeprecationWarning)


ip_graph = nx.DiGraph(id='IPGraph')
_, ip_entities = iplocation.lookup_ip(ip_address=target_ip)
if ip_entities:
    ip_entity = ip_entities[0]
    ip_desc = f'{target_ip}\n{ip_entity.Location.City}, {ip_entity.Location.CountryName}'
else:
    ip_desc = 'unknown location'
ip_graph.add_node(target_ip, name=target_ip, description=ip_desc, node_type='ip')

def add_vm_node(graph, source, row):
    vm_desc = f'{row.VMName}\n{row.ResourceGroup}, {row.VMRegion}'
    ip_graph.add_node(row.VMName, name=row.VMName, description=vm_desc,
                      node_type='host')
    if row.FlowDirection == 'I':
        ip_graph.add_edge(target_ip, row.VMName)
    else:
        ip_graph.add_edge(row.VMName, target_ip)
#     print('Adding ', target_ip, row['VMName'])
    
# Cycle through entities
az_net_ext_comms_df.apply(lambda x: add_vm_node(ip_graph, target_ip, x),axis=1)

src_node = [n for (n, node_type) in
            nx.get_node_attributes(ip_graph, 'node_type').items()
            if node_type == 'ip']
vm_nodes = [n for (n, node_type) in
            nx.get_node_attributes(ip_graph, 'node_type').items()
            if node_type == 'host']

# now draw them in subsets  using the `nodelist` arg
plt.rcParams['figure.figsize'] = (10, 10)
plt.margins(x=0.3, y=0.3)
plt.title('Internal hosts ')
pos = nx.circular_layout(ip_graph)
nx.draw_networkx_nodes(ip_graph, pos, nodelist=src_node,
                       node_color='red', alpha=0.5, node_shape='o')
nx.draw_networkx_nodes(ip_graph, pos, nodelist=vm_nodes,
                       node_color='green', alpha=0.5, node_shape='s',
                       s=400)
nlabels = nx.get_node_attributes(ip_graph, 'description')
nx.relabel_nodes(ip_graph, nlabels)
nx.draw_networkx_labels(ip_graph, pos, nlabels, font_size=15)
nx.draw_networkx_edges(ip_graph, pos, alpha=0.5, arrows=True, arrowsize=20);


In [ ]:
# reverse DNS lookup
from dns import reversename, resolver
rev_name = reversename.from_address(src_ip_entity.Address)

print(rev_name)
try:
    rev_dns = str(resolver.query(rev_name, 'PTR'))
    rev_dns
except:
    print('no reverse addr result')

from ipwhois import IPWhois
whois = IPWhois(src_ip_entity.Address)

whois_result = whois.lookup_whois()
whois_result


<a id='threatintel'></a>[Contents](#toc)
# Threat Intel Lookups

In [ ]:
threat_intel_qry = '''
'''

<a id='virustotallookup'></a>[Contents](#toc)
## Virus Total Lookup

In [ ]:
vt_key = asi.GetEnvironmentKey(env_var='VT_API_KEY',
                           help_str='To obtain an API key sign up here https://www.virustotal.com/',
                           prompt='Virus Total API key:')
vt_key.display()

In [ ]:
if vt_key.value:
    vt_lookup = sectools.VTLookup(vt_key.value, verbosity=2)

    print(f'{len(ioc_df)} items in input frame')
    supported_counts = {}
    for ioc_type in vt_lookup.supported_ioc_types:
        supported_counts[ioc_type] = len(ioc_df[ioc_df['IoCType'] == ioc_type])
    print('Items in each category to be submitted to VirusTotal')
    print('(Note: items have pre-filtering to remove obvious erroneous '
          'data and false positives, such as private IPaddresses)')
    print(supported_counts)
    print('-' * 80)
    vt_results = vt_lookup.lookup_iocs(data=ioc_df, type_col='IoCType', src_col='Observable')
    display(vt_results)

<a id='related_alerts'></a>[Contents](#toc)
# Related Alerts
For a subset of entities in the alert we can search for any alerts that have that entity in common. Currently this query looks for alerts that share the same Host, Account or Process and lists them below. 
**Notes:**
- Some alert types do not include all of these entity types.
- The original alert will be included in the "Related Alerts" set if it occurs within the query time boundary set below.

The query time boundaries default to a longer period than when searching for the alert. You can extend the time boundary searched before or after the alert time. If the widget doesn't support the time boundary that you want you can change the max_before and max_after parameters in the call to QueryTime below to extend the possible time boundaries.

In [ ]:
# set the origin time to the time of our alert
query_times = asi.QueryTime(units='day', origin_time=security_alert.TimeGenerated, 
                            max_before=28, max_after=1, before=5)
query_times.display()

In [ ]:
related_alerts = qry.list_related_alerts(provs=[query_times, security_alert])

host_alert_items = related_alerts\
    .query('host_match == @True')[['AlertType', 'StartTimeUtc']]\
    .groupby('AlertType').StartTimeUtc.agg('count').to_dict()
acct_alert_items = related_alerts\
    .query('acct_match == @True')[['AlertType', 'StartTimeUtc']]\
    .groupby('AlertType').StartTimeUtc.agg('count').to_dict()
proc_alert_items = related_alerts\
    .query('proc_match == @True')[['AlertType', 'StartTimeUtc']]\
    .groupby('AlertType').StartTimeUtc.agg('count').to_dict()

def print_related_alerts(alertDict, entityType, entityName):
    if len(alertDict) > 0:
        print('Found {} different alert types related to this {} (\'{}\')'.format(len(alertDict), entityType, entityName))
        for (k,v) in alertDict.items():
            print('    {}, Count of alerts: {}'.format(k, v))
    else:
        print('No alerts for {} entity \'{}\''.format(entityType, entityName))
        
print_related_alerts(host_alert_items, 'host', security_alert.hostname)
print_related_alerts(acct_alert_items, 'account', 
                     security_alert.primary_account.qualified_name if security_alert.primary_account
                     else None)
print_related_alerts(proc_alert_items, 'process', 
                     security_alert.primary_process.ProcessFilePath if security_alert.primary_process
                     else None)

### Show these related alerts on a graph
This should indicate which entities the other alerts are related to.

This can be unreadable with a lot of alerts. Use the matplotlib interactive zoom control to zoom in to part of the graph.

In [ ]:
# Draw a graph of this (add to entity graph)
%matplotlib notebook
%matplotlib notebook
alertentity_graph = asi.create_alert_graph(security_alert)
rel_alert_graph = asi.add_related_alerts(related_alerts=related_alerts,
                                         alertgraph=alertentity_graph)
asidisp.draw_alert_entity_graph(rel_alert_graph, width=15)

### Browse List of Related Alerts
Select an Alert to view details. 

If you want to investigate that alert - copy its *SystemAlertId* property and open a new instance of this notebook to investigate this alert.

In [ ]:
related_alerts['CompromisedEntity'] = related_alerts['Computer']

def disp_full_alert(alert):
    global related_alert
    related_alert = asi.SecurityAlert(alert)
    asidisp.display_alert(related_alert, show_entities=True)

print('Selected alert is available as \'related_alert\' variable.')
rel_alert_select = asi.AlertSelector(alerts=related_alerts, action=disp_full_alert)
rel_alert_select.display()


<a id='appendices'></a>[Contents](#toc)
# Appendices

## Available DataFrames

In [ ]:
print('List of current DataFrames in Notebook')
print('-' * 50)
current_vars = list(locals().keys())
for var_name in current_vars:
    if isinstance(locals()[var_name], pd.DataFrame) and not var_name.startswith('_'):
        print(var_name)

## Saving Data to Excel
To save the contents of a pandas DataFrame to an Excel spreadsheet
use the following syntax
```
writer = pd.ExcelWriter('myWorksheet.xlsx')
my_data_frame.to_excel(writer,'Sheet1')
writer.save()
```